In [ ]:
# Install required libraries
!pip install gradio opencv-python-headless imutils tensorflow numpy

In [1]:
# Mount Google Drive to access the model file
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2
import numpy as np
import gradio as gr
import imutils
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model

In [ ]:
# Download Haar cascade file
!wget https://github.com/opencv/opencv/raw/master/data/haarcascades/haarcascade_frontalface_default.xml

--2025-06-10 19:31:15--  https://github.com/opencv/opencv/raw/master/data/haarcascades/haarcascade_frontalface_default.xml
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml [following]
--2025-06-10 19:31:15--  https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 930127 (908K) [text/plain]
Saving to: ‘haarcascade_frontalface_default.xml’

haarcascade_frontal 100%[===================>] 908.33K  3.42MB/s    in 0.3s    

20

In [ ]:
# Parameters for loading data and models
detection_model_path = 'haarcascade_frontalface_default.xml'
emotion_model_path = '/content/drive/MyDrive/Capstone DL project/final_mini_XCEPTION_model_v2.keras'

In [ ]:
# Load models
try:
    face_detection = cv2.CascadeClassifier(detection_model_path)
    if not face_detection.empty():
        print("Haar cascade loaded successfully")
    else:
        raise ValueError("Failed to load Haar cascade file")
except Exception as e:
    print(f"Error loading Haar cascade: {e}")

try:
    emotion_classifier = load_model(emotion_model_path, compile=False)
    print("Emotion model loaded successfully")
except Exception as e:
    print(f"Error loading emotion model: {e}")
    raise

Haar cascade loaded successfully
Emotion model loaded successfully


In [ ]:

# Define emotions (ensure these match your model's output classes)
EMOTIONS = ["angry", "disgust", "scared", "happy", "sad", "surprised", "neutral"]


In [ ]:
def process_frame(frame):
    try:
        # Ensure frame is in correct format (numpy array, BGR)
        if frame is None:
            return None, "No frame received"

        # Resize and convert frame
        frame = imutils.resize(frame, width=300)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frameClone = frame.copy()
        canvas = np.zeros((250, 300, 3), dtype="uint8")

        # Detect faces
        faces = face_detection.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30), flags=cv2.CASCADE_SCALE_IMAGE)

        label = "No face detected"
        if len(faces) > 0:
            # Sort faces by size and take the largest one
            faces = sorted(faces, reverse=True, key=lambda x: (x[2] - x[0]) * (x[3] - x[1]))[0]
            (fX, fY, fW, fH) = faces

            # Extract and preprocess ROI
            roi = gray[fY:fY + fH, fX:fX + fW]
            roi = cv2.resize(roi, (48, 48))  # Changed to 48x48 to match model input
            roi = roi.astype("float") / 255.0
            roi = img_to_array(roi)
            roi = np.expand_dims(roi, axis=0)

            # Predict emotions
            preds = emotion_classifier.predict(roi)[0]
            emotion_probability = np.max(preds)
            label = EMOTIONS[preds.argmax()]

            # Draw probability bars
            for (i, (emotion, prob)) in enumerate(zip(EMOTIONS, preds)):
                text = "{}: {:.2f}%".format(emotion, prob * 100)
                w = int(prob * 300)
                cv2.rectangle(canvas, (7, (i * 35) + 5), (w, (i * 35) + 35), (0, 0, 255), -1)
                cv2.putText(canvas, text, (10, (i * 35) + 23), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255, 255, 255), 2)

            # Draw face rectangle and label
            cv2.putText(frameClone, label, (fX, fY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
            cv2.rectangle(frameClone, (fX, fY), (fX + fW, fY + fH), (0, 0, 255), 2)

        # Combine frame and canvas for display
        output = np.vstack([frameClone, canvas])
        return output, label
    except Exception as e:
        return frame, f"Error processing frame: {str(e)}"

In [ ]:
# Gradio interface
iface = gr.Interface(
    fn=process_frame,
    inputs=gr.Image(type="numpy", label="Webcam Input"),  # Removed 'source' parameter
    outputs=[gr.Image(type="numpy", label="Processed Frame"), gr.Textbox(label="Detected Emotion")],
    live=True,  # Enables real-time streaming
    title="Facial Emotion Detection",
    description="This app detects emotions from a Mini XCEPTION model. Ensure your webcam is enabled."
)

In [ ]:
# Launch the interface
iface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://54a643f5ce766de351.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:

# ### Steps to Run in Colab:
# 1. **Mount Google Drive**: When you run the code, Colab will prompt you to mount your Google Drive. Follow the instructions to authenticate and mount it, ensuring the model file is accessible at `/content/drive/MyDrive/Capstone DL project/best_mini_XCEPTION.keras`.
# 2. **Verify Model Compatibility**:
#    - **Input Shape**: Check your model's architecture to confirm it expects 64x64 grayscale images. You can inspect the model with `emotion_classifier.summary()` after loading it. If the input shape is different (e.g., 48x48), update the line `roi = cv2.resize(roi, (64, 64))` to match (e.g., `roi = cv2.resize(roi, (48, 48))`).
#    - **Output Classes**: Ensure the model outputs 7 classes corresponding to `EMOTIONS`. If the number or order of classes differs, update the `EMOTIONS` list to match your model's output layer.
# 3. **Run the Code**: Execute the cells in Colab. The Haar cascade file will be downloaded automatically, and your model will be loaded from Google Drive.
# 4. **Gradio Interface**: After launching, Gradio will provide a public URL (or local interface) to access the webcam feed. Ensure your browser allows webcam access.
# 5. **Troubleshooting**:
#    - If the model fails to load, check the file path and ensure the `.keras` file is not corrupted.
#    - If you get a `ValueError` about custom objects, you may need to define any custom layers or metrics used during training. For example, if your model uses a custom loss, you might need to load it with `custom_objects={'loss_name': loss_function}`.
#    - If no faces are detected, adjust the `detectMultiScale` parameters (e.g., lower `minNeighbors` or `scaleFactor`) for better detection.

# ### Potential Issues and Fixes:
# - **Model Input Mismatch**: If your model expects a different input shape, the code will fail during prediction. Check the model’s input shape and adjust the resize step accordingly.
# - **Class Mismatch**: If your model was trained on fewer or different emotions, the `EMOTIONS` list and indexing will cause errors. Update the list to match your model’s classes.
# - **Google Drive Access**: Ensure the model file is in the specified path. If it’s in a different folder, update `emotion_model_path`.
# - **Gradio Webcam Issues**: Colab’s webcam access via Gradio can be finicky. If the webcam doesn’t work, ensure your browser permissions allow camera access, or test with a local Python environment if possible.

# ### Testing the Model:
# To verify your model’s compatibility before running the full Gradio interface, you can add this snippet after loading the model:

# ```python
# # Check model summary
# emotion_classifier.summary()

# # Test with a dummy input
# dummy_input = np.zeros((1, 64, 64, 1))  # Adjust shape if needed
# try:
#     preds = emotion_classifier.predict(dummy_input)
#     print(f"Model output shape: {preds.shape}")
#     print(f"Predicted probabilities: {preds}")
# except Exception as e:
#     print(f"Error with model prediction: {e}")
# ```

# This will confirm the model’s input/output shapes and help diagnose issues.

# ### Additional Notes:
# - The code retains the same functionality as the original: it detects faces, predicts emotions, draws bounding boxes, and displays probability bars.
# - The commented-out emoji overlay from the original code is omitted for simplicity, as it requires additional image files. If you want to include it, upload the emoji images to your Google Drive and add the overlay logic back (uncomment and adapt the relevant section).
# - If you encounter performance issues in Colab (e.g., slow processing), consider optimizing the frame rate or using a local machine with a GPU for better performance.

# Let me know if you need help verifying your model’s specifications, debugging errors, or adding the emoji overlay!